---
# 3D Wing Motion Kinematics

---

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import trapz

#### Marin Lauber 2020

The ellipsoidal wing is defined as

\begin{equation}
    \frac{x^2}{a^2} + \frac{y^2}{b^2} + \frac{z^2}{c^2} = 1,
\end{equation}

with $a=0.5$, $b=0.05$ and $c=1$. 

In [8]:
# wing
def chord(r):
    return np.sqrt(abs((1-r**2)))
def c(r, Rtip=2.5):
    return np.where(r<(Rtip-2.0), 0, chord(r-1.5))

Its motion is deine by three Euler angles

\begin{split}
    \varphi(t) &= A_\varphi\cos(2\pi f_r t), \\
    \theta(t) &= A_\theta\sin(2\pi f_\theta t), \\
    \alpha(t) &= \frac{\pi}{2} - A_\alpha\sin(2\pi f_r t + \xi),
\end{split}

where $\varphi(t)$ corresponds to the stroke variation, $\alpha(t)$ to the geometrical angle of attack and $\theta(t)$ to the deviation from the horizontal stroke plane. $\xi$ is the phase difference between $\alpha$ and $\theta$. Figure-0 motion are obtained by setting $f_\theta = f_r$ and figure-of-8 by setting $f_\theta = 2f_r$

In [9]:
# elevation angle
def theta(t, A, f): 
    return A*np.cos(2*np.pi*f*t)
def thetadot(t, A, f):
    return -2*np.pi*f*A*np.sin(2*np.pi*f*t)

# stroke angle
def phi(t, A, f): 
    return A*np.sin(2*np.pi*f*t)
def phidot(t, A, f):
    return 2*np.pi*f*A*np.cos(2*np.pi*f*t)

# angle of attack
def alpha(t, A, f, xi=0.): 
    return np.pi/2. - A*np.cos(2*np.pi*f*t + xi)
def alphadot(t, A, f, xi=0):
    return -2*np.pi*f*A*np.sin(2*np.pi*f*t + xi)

The aerodynamic performance of the flapping wing is evaluated using the drag coefficient $C_D$ and lift coefficient $C_L$

\begin{split}
    C_D &= \frac{F_x\cos(\varphi) - F_z\sin(\varphi)}{\frac{1}{2}\rho U_{ref}^2S}, \\
    C_L &= \frac{F_y}{\frac{1}{2}\rho U_{ref}^2S}. \\
\end{split}

The wing area is $S=\pi/4.$ and the reference velocity is defined as (average velocity magnitude over one period)

\begin{equation}
    U_{ref} = f_r\int_{0}^{1/f_r}\sqrt{\dot{\varphi}^2 + \dot{\theta}^2}dt R_g \qquad\left( = f_r\int_{0}^{1/f_r}|\dot{\varphi}| dt R_g\right).
\end{equation}

When $\theta(t)\equiv0$.

\begin{equation}
U_{ref} = 2\pi f_r A_\varphi f_r\int_{0}^{1/f_r}|\sin(2\pi f_r t)| dt R_g =0.5 \pi A_\varphi \int_{0}^{2}|\sin(\pi t)| dt R_g \sim 0.5 \pi A_\varphi 1.2732 R_g
\end{equation}
The radius of gyration is

\begin{equation}
    R_g = \sqrt{\frac{1}{S}\int_{0}^{R}cr^2 dr}.
\end{equation}

Both those integrals are better carried numerically.

In [10]:
# kinematic params
fr = 0.5
t, dt = np.linspace(0.0, 1./fr, 512, retstep=True)
r, dr = np.linspace(0, 2.5, 512, retstep=True)
A_phi = 0.35*np.pi
S = np.pi/2.
cbar = np.pi/4.
nu = 0.0273

In [11]:
# integrals
Rg = np.sqrt(1./S*trapz(c(r)*r**2, r, dr))
Uref = fr * trapz(abs(phidot(t, A_phi, fr)), t, dt) * Rg   

In [17]:
# printing stuff
print('Radius of gyration : %.3f' % Rg)
print('Uref : %.3f' % Uref)

Radius of gyration : 1.581
Uref : 3.477


The Reynolds number is defined as

\begin{equation}
    Re = \frac{\bar{c}U_{ref}}{\nu} = \frac{4A_\phi f_r R_g \bar{c}}{\nu}
\end{equation}

where $\nu=0.0273$ and the average cord length is $\bar{c} = \pi/4.$

In [13]:
# Reynolds number
def Re(Uref, cbar, nu, A_phi, fr, Rg):
    return Uref*cbar/nu, 4*A_phi*fr*Rg*cbar/nu

In [14]:
print('Reynolds number : %.1f (%.1f)' % Re(Uref, cbar, nu, A_phi, fr, Rg))

Reynolds number : 100.0 (100.0)
